In [32]:
from sklearn import datasets
import pandas as pd
import numpy as np
import datetime

In [33]:
#初始化参数
def init(n_x,n_h,n_y):
    np.random.seed(2)#设置随机种子
    #权重和偏置矩阵
    w1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros(shape=(n_h,1))
    w2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros(shape=(n_y,1))
    
    parameters={'w1':w1,'b1':b1,'w2':w2,'b2':b2}
    return parameters

In [34]:
#前向传播
def forward_propagation(X,parameters):
    w1=parameters['w1']
    b1=parameters['b1']
    w2=parameters['w2']
    b2=parameters['b2']
    
    #计算a2
    z1=np.dot(w1,X)+b1
    a1=np.tanh(z1)    #第一层激活函数
    z2=np.dot(w2,a1)+b2
    a2=1/(1+np.exp(-z2))#sigmoid 第二层激活函数
    
    cache={'z1':z1,'a1':a1,'z2':z2,'a2':a2}
    
    return a2,cache

In [35]:
#计算代价函数
def compute_cost(a2,Y,parameters):
    m=Y.shape[1] #Y的列数为总样本数
    
    #交叉熵作为代价函数
    logprobs=np.multiply(np.log(a2),Y)+np.multiply((1-Y),np.log(1-a2))
    cost=-np.sum(logprobs)/m
    return cost

In [36]:
#反向传播
def backward_propagation(parameters,cache,X,Y):
    m=Y.shape[1]
    
    w2=parameters['w2']
    
    a1=cache['a1']
    a2=cache['a2']
    
    dz2=a2-Y
    dw2=(1/m)*np.dot(dz2,a1.T)
    db2=(1/m)*np.sum(dz2,axis=1,keepdims=True)
    dz1=np.multiply(np.dot(w2.T,dz2),1-np.power(a1,2))
    dw1=(1/m)*np.dot(dz1,X.T)
    db1=(1/m)*np.sum(dz1,axis=1,keepdims=True)
    
    grads={'dw1':dw1,'db1':db1,'dw2':dw2,'db2':db2}
    
    return grads

In [37]:
#更新参数
def update_parameters(parameters,grads,learning_rate=0.4):
    w1=parameters['w1']
    b1=parameters['b1']
    w2=parameters['w2']
    b2=parameters['b2']
    
    dw1=grads['dw1']
    db1=grads['db1']
    dw2=grads['dw2']
    db2=grads['db2']
    
    w1=w1-dw1*learning_rate
    b1=b1-db1*learning_rate
    w2=w2-dw2*learning_rate
    b2=b2-db2*learning_rate
    
    parameters={'w1':w1,'b1':b1,'w2':w2,'b2':b2}
    
    return parameters

In [38]:
#建立神经网络
def nn_model(X,Y,n_h,n_input,n_output,num_iterations=10000,print_cost=False):
    np.random.seed(3)
    n_x=n_input#输入层节点数
    n_y=n_output#输出层节点数
    
    parameters=init(n_x,n_h,n_y)
    
    #梯度下降
    for i in range(0,num_iterations):
        a2,cache=forward_propagation(X,parameters)
        cost=compute_cost(a2,Y,parameters)
        grads=backward_propagation(parameters,cache,X,Y)
        parameters=update_parameters(parameters,grads)
        
        if print_cost and i%1000==0:
            print('迭代第%i次，代价函数为%f'%(i,cost))
    return parameters

In [60]:
def predict(parameters,x_test,y_test):
    w1=parameters['w1']
    b1=parameters['b1']
    w2=parameters['w2']
    b2=parameters['b2']
    
    z1=np.dot(w1,x_test)+b1
    a1=np.tanh(z1)
    z2=np.dot(w2,a1)+b2
    a2=1/(1+np.exp(-z2))
    
    n_rows=y_test.shape[0]
    n_cols=y_test.shape[1]
    
    output=np.empty(shape=(n_rows,n_cols),dtype=int)
    
    for i in range(n_rows):
        for j in range(n_cols):
            if a2[i][j]>0.5:
                output[i][j]=1
            else:
                output[i][j]=0
    
    print("预测结果：")
    print(output)
    print("真实结果")
    print(y_test)
    
    count=0
    for k in range(0,n_cols):
        if(output[0][k]==y_test[0][k] and output[1][k]==y_test[1][k] and output[2][k]==y_test[2][k]):
            count=count+1
        else:
            print(k)
    acc=count / int(y_test.shape[1])*100
    print("准确率：%.2f%%"%acc)

In [61]:
if __name__=="__main__":
    data_set=datasets.load_iris()
    xx=data_set.data
    X=xx[0:125]
    x_test=xx[125:]
    X=X.T
    x_test=x_test.T
    yy=data_set.target
    Y=[[0]*3]*len(yy)
    Y=np.array(Y)
    for i in range(len(yy)):
        if yy[i]==0:
            Y[i][0]=1
            Y[i][1]=0
            Y[i][2]=0
        elif yy[i]==1:
            Y[i][0]=0
            Y[i][1]=1
            Y[i][2]=0
        else:
            Y[i][0]=0
            Y[i][1]=0
            Y[i][2]=1
    YY=Y[0:125]
    y_test=Y[125:]
    YY=YY.T
    y_test=y_test.T
    YY=YY.astype('uint8')
    y_test=y_test.astype('uint8')
    #start
    start_time=datetime.datetime.now()
    #输入层4个节点，隐藏层10个节点，输出层3个节点，迭代10000次
    parameters=nn_model(X,YY,n_h=10,n_input=4,n_output=3,num_iterations=10000,print_cost=True)
    end_time=datetime.datetime.now()
    print("用时："+str((end_time-start_time).seconds)+str(round((end_time-start_time).microseconds/1000))+"ms")
    
    predict(parameters,x_test,y_test)

迭代第0次，代价函数为2.079440
迭代第1000次，代价函数为0.141475
迭代第2000次，代价函数为0.077525
迭代第3000次，代价函数为0.052129
迭代第4000次，代价函数为0.284684
迭代第5000次，代价函数为0.058633
迭代第6000次，代价函数为0.061312
迭代第7000次，代价函数为0.043087
迭代第8000次，代价函数为0.003837
迭代第9000次，代价函数为0.002737
用时：1606ms
预测结果：
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]]
真实结果
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
0
2
4
6
8
9
13
准确率：72.00%
